# FORCE ESTIMATION PREPROCESSING

## Specify Paths and FLAGS

In [12]:
## TODO
# - unit convert to do but raw embedded into SI units (Si Machine) - DONE
# - jacobian preprocessing using interpolate jacobian (Si Machine) - DONE
# - split embeded data into force sensor ground truth (Si Machine ) - DONE
# - model prediction
# - procesing rosbag data through simons repo dvrk_force_estimation repo (Si Machine )
# - - rosbag reprocess (Si Machine)
# - - preprocess script(interpolations) (Si Machine)


# NOTES
#   make sure you put the json converted files in the jhu-davinci folders  

# file struture
# ---------------
# dataset_name 
# |__joints


## EMBEDDED PATHS
dataset_path = "/home/ndrakes1/force_estimation_data_captures/capture_9-5-2025/test_preprocess/" # SET
dataset_name = "bias_joint1"

fpga_v3_data_collection_path = "/home/ndrakes1/fpgav3-data-collection" # SET THIS ONCE
preproces_path = " "  # should be this directory SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_script_py = fpga_v3_data_collection_path + "/unit_convert/unit_convert.py"
si_unit_xml_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM3-698534.xml" # SET THIS ONCE 
si_unit_json_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM1-334809.xml.json"

zynq_data_csv = "/home/ndrakes1/force_estimation_data_captures/capture_9-5-2025/test_preprocess/bias_joint1.csv" 
rosbag_data_bag = "/home/ndrakes1/force_estimation_data_captures/capture_9-5-2025/test_preprocess/bias_joint1.bag"

unit_convert_path = dataset_path + "/" + dataset_name + "_unitConvert.csv"

# unit_convert_cmd = "python3 " + unit_convert_path + " -c " + si_unit_json_path + " -f " 


# ROSBAG PATHS
read_rosbag_script_py = "/home/ndrakes1/dvrk_force_estimation/preprocess/read_ros_bags.py"
preprocess_rosbag_script_py = "/home/ndrakes1/dvrk_force_estimation/preprocess/preprocess.py"


ORIGINAL_FREQ = 1000

INTERPOLATE = True


FILTER = False
FILTER_FREQ = 100
FILTER_VELOCITY = True

DOWNSAMPLE = False
DOWNSAMPLE_FREQ = 1000
DOWNSAMPLE_MOVING_AVERAGE = True

##################################################################################

# EMBEDDED

In [13]:
!. ~/catkin_ws/devel/setup.bash

## Unit Convert

In [14]:
!.venv/bin/python3 {unit_convert_script_py} -c {si_unit_json_path} -f {zynq_data_csv}

Traceback (most recent call last):
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 115, in <module>
    main()
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 110, in main
    dataConverted = unitConvert(args.configuration, args.file_name)
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 13, in unitConvert
    with open(configuration, 'r') as file:
FileNotFoundError: [Errno 2] No such file or directory: '/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM1-334809.xml.json'


## Extract Force Sensor Data

In [15]:
!mkdir {dataset_path}/sensor
!.venv/bin/python3 sensor_processing.py {zynq_data_csv} {dataset_path}/sensor/sensor.csv

mkdir: cannot create directory ‘/home/ndrakes1/force_estimation_data_captures/capture_9-5-2025/test_preprocess//sensor’: File exists
Saved SENSOR DATA CSV to /home/ndrakes1/force_estimation_data_captures/capture_9-5-2025/test_preprocess//sensor/sensor.csv


## Extract Joint Data

In [16]:
!mkdir {dataset_path}joints
!.venv/bin/python3 preprocessing.py {unit_convert_path} {dataset_path}joints/interpolated_all_joints.csv

mkdir: cannot create directory ‘/home/ndrakes1/force_estimation_data_captures/capture_9-5-2025/test_preprocess/joints’: File exists


## Filter Joint Data

### Preprocessed Training File Paths

In [17]:
import os
import pandas as pd

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")
df = pd.read_csv(data_file, header=None)


### Interpolate

In [18]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):
    
    df = interpolate_dataframe_to_sample_rate(df, ORIGINAL_FREQ)



[0.00000000e+00 4.88002000e-04 1.48277200e-03 ... 3.81164781e+01
 3.81174781e+01 3.81184781e+01]


### FIR Filtering

In [19]:
import importlib
import filter  # regular import first
importlib.reload(filter)


if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to data
    df = filter.apply_filter_to_torque_feedback_df(df, fir_coeffs, filter_velocity=FILTER_VELOCITY)

### Downsample


In [20]:
from downsample import downsample_dataframe

if (DOWNSAMPLE == True):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


### SAVE TO CSV

In [21]:
df.to_csv(data_file, index=False, header=False)


## Extract Jacobian

In [22]:
!mkdir {dataset_path}/jacobian
!.venv/bin/python3 interpolate_jacobian2.py {unit_convert_path} {dataset_path}/jacobian/interpolated_all_jacobian.csv dvpsm.rob

mkdir: cannot create directory ‘/home/ndrakes1/force_estimation_data_captures/capture_9-5-2025/test_preprocess//jacobian’: File exists
[[ 8.67361738e-18 -2.42861287e-17  4.42886203e-01 -2.86229374e-17
  -8.73922559e-02 -3.70352717e-02]
 [-1.73472348e-18  2.08166817e-17  7.77465986e-01  1.30104261e-17
   5.67102209e-02 -3.59496939e-02]
 [-1.38777878e-17  2.08166817e-17 -4.46540537e-01  3.46944695e-18
   1.20603684e-02 -9.93237374e-02]
 [ 5.55111512e-17 -7.10006036e-01  0.00000000e+00  4.42886203e-01
  -3.30867357e-01  9.04036968e-01]
 [-1.00000000e+00 -5.55111512e-17  0.00000000e+00  7.77465986e-01
  -3.21168974e-01 -3.77545036e-01]
 [ 1.38777878e-16 -7.04195590e-01  0.00000000e+00 -4.46540537e-01
  -8.87342821e-01 -2.00441776e-01]]


# NOW DO INFERENCE ON THE JOINT DATA ON THE DVRK-12-MACHINE